In [1]:
import sys, pathlib, itertools
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20, 10)

In [2]:
# We need to clone project and add it to sys.path
# TODO git clone
# TODO install required packages

print(pathlib.Path('..\..').resolve())
sys.path.append(str(pathlib.Path('..\..').resolve()))

D:\dieyepy\src\main\python


In [3]:
# slicing - 
# patch_label_image
# patch_pos
# patch_geometry
# 

# There are several use-cases for slicing slide-images.
# 1) Generating patch_label_images together with generating patch_slide_images. Where
#    a) patch_label_image is a result of drawing annotation with label color stored in annotation
#    b) patch_slide_image is just a corresponding region from the same slide
# 2) Generate patch_label_images together with generating patch_slide_images. Where
#    a) patch_label_image is a region from some specifically stained slide
#    b) patch_slide_image is the same region from (possibly a bit shifted) corresponding H&E slide
# 3) Do not generate patch_label_images but generate just patch_slide_images.
#    It can be useful when you want:
#    - label patches manually in some graphics editor
#    - label patches with your custom script
# 
# Here we will consider the use-case 1.
# We will generate patch_label_images and then generate corresponding patch_slide_images.
# 
# High-level api for generating patches operates on dict-like config objects.
# There are 2 types of config: PatchImageSourceConfig, PatchImageConfig.
# PatchImageSourceConfig defines params for generating patches from slide.
# It is named "source" because it initiates data-flow:
# patch_pos->patch_geometry->patch_label_image->patch_slide_image.
# It is designed so to allow hooks.
# Different hooks can be placed in this data-flow such that irrelevant 
# and uninteresting positions/resulting label images will be filtered out.
# At this moment PatchResponseGenerator use such hooks that 
# it skips patch geometry if it doesn't fully contained by some roi annotation.
# You can add your hooks to this data-flow. For example you can add hook to filter out patch label images
# that are uninteresting in sense of class distribution - ignore fully
# white or fully black resuting label images but take only label image containing both classes.
# 
# Grid positions and label images can be filtered out in data-flow so 
# ONLY AFTER generating patch_label_image and confirming it as "interesting"(means not filtering it out)
# it make sense to generate corresponding patch_slide_image.
# It means that process of generating patch_slide_images depends on process 
# of generating patch_label_images. This dependence is represented with attribute 
# "dependents" in PatchImageSourceConfig.
# 
# The configs defined below specify to:
# generate patch_label_images from slide1 based on slide1_annotations
# generate corresponding(geometrically) patch_slide_images from slide1
# generate patch_label_images from slide2 based on slide2_annotations
# generate corresponding(geometrically) patch_slide_images from slide2

In [4]:
# grid length in pixels. Resulting patches will have size (grid_length,grid_length)
grid_length = 512
# level of slide. Can be interpreted as level of detail(resolution) with the best detail at 0-level.
level = 2
# stride of slicer - distance between subsequent patches
stride = grid_length / 2

In [5]:
from slice.model.patch_image_config import PatchImageConfig
from slice.model.patch_image_source_config import PatchImageSourceConfig
configs = [
    PatchImageSourceConfig(
        r"D:\temp\slides\slide1.mrxs",
        level,
        rescale_result_image=True,
        annotations_path=r"D:\temp\slides\slide1_annotations.json",
        metadata={"name": "label"}, 
        grid_length=grid_length, 
        stride=stride,
        dependents=[
            PatchImageConfig(
                r"D:\temp\slides\slide1.mrxs",
                level,
                rescale_result_image=True,
                metadata={"name": "image"}
            )
        ]
    ),
    PatchImageSourceConfig(
        r"D:\temp\slides\slide5.mrxs",
        level,
        rescale_result_image=True,
        annotations_path=r"D:\temp\slides\slide5_annotations.json",
        metadata={"name": "label"},
        grid_length=grid_length,
        stride=stride,
        dependents=[
            PatchImageConfig(
                r"D:\temp\slides\slide5.mrxs",
                level,
                rescale_result_image=True,
                metadata={"name": "image"}
            )
        ]
    )
]

In [6]:
# PatchResponseGenerator processes configs one by one generating patches and puts these patches into 
# one data-flow: Iterable[PatchResponse]
from slice.generator.response.patch_response_generator import PatchResponseGenerator
patch_responses = PatchResponseGenerator().create(configs)

In [7]:
# Then we convert every patch_response to named_ndarray - Tuple[str, np.ndarray]
# We select name format convenient for saving/loading to/from disk.
from slice.patch_response_utils import patch_responses_to_named_ndarrays
format_str = r"{cfg.slide_path}/{cfg.grid_length}/{cfg.metadata[name]}/({pos[1]},{pos[0]})_{cfg.level}_{cfg.metadata[name]}.png"
named_ndarrays = patch_responses_to_named_ndarrays(patch_responses, format_str)

In [8]:
# Lets collect data-flow to list and print some info 
def print_named_ndarrays_info(named_ndarrays):
    print(f"arrays count: {len(named_ndarrays)}")
    print(f"1-st array (name,shape): {(named_ndarrays[0][0], named_ndarrays[0][1].shape)}")
# named_ndarrays=itertools.islice(named_ndarrays, 100)
named_ndarrays = list(named_ndarrays)
print_named_ndarrays_info(named_ndarrays)

Ignoring not polygon or rect geometry: annotation_type=<AnnotationType.ELLIPSE: 3> origin_point=(0, 0) points=[(33490, 141253), (33994, 141745)]
Ignoring not polygon or rect geometry: annotation_type=<AnnotationType.ELLIPSE: 3> origin_point=(0, 0) points=[(33490, 141253), (33994, 141745)]
arrays count: 2814
1-st array (name,shape): ('D:/temp/slides/slide1.mrxs/512/label/(63232,41728)_2_label.png', (1, 512, 512, 3))


In [9]:
# We often want to store results of generating patches.
# It is convenient to store data hierarchically.
# 3 hierarchial data storages are supported:
# 1) hdf5 (patches are stored as arrays in a single file)
# 2) zip archive (patches are stored as arrays in a single file)
# 3) file system folders (patches are stored as image files in folders)
# 
# Single file with arrays VS folders with image files.
# Single file with arrays is easier and faster to:
# a) manage(delete, copy)
# b) transfer and synchronize with remote storage (like google disk which can be used from google colab)
# c) use in processing later (for examplle training model with keras)
# File system folder with image files is easier to:
# a) explore
# b) image-compression often is better than zip-compression for arrays
# 
data_folder = pathlib.Path.home().joinpath("temp/slice")
# Choose data storage type (one of hdf5, zip, folder)
# data_path = pathlib.Path(data_folder, 'data.hdf5')
# data_path = pathlib.Path(data_folder, 'data.zip')
data_path = data_folder
print(data_path)

C:\Users\User\temp\slice


In [10]:
from ndarray_persist.ndarray_persist_utils import save_named_ndarrays
save_named_ndarrays(named_ndarrays, data_path, delete_if_exists=True, verbosity=1)

saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(63232,41728)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(63232,41984)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(63232,42240)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(63232,42496)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(63232,42752)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(63488,41728)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(63488,41984)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(63488,42240)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(63488,42496)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(63488,42752)_2_label.png


saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(65280,52992)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(65280,53248)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(65280,53504)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(65280,53760)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(65280,54016)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(65536,50688)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(65536,50944)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(65536,51200)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(65536,51456)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(65536,51712)_2_label.png


saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(72448,45824)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(72448,46080)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(72448,46336)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(72448,46592)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(72448,46848)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(72448,47104)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(72448,57600)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(72448,57856)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(72448,58112)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(72448,58368)_2_label.png


saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(73984,57600)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(73984,57856)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(73984,58112)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(73984,58368)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(73984,58624)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(73984,58880)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(73984,59136)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(73984,59392)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(74240,39168)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(74240,39424)_2_label.png


saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(75264,47872)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(75264,48128)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(75264,48384)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(75264,48640)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(75264,48896)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(75264,49152)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(75520,39168)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(75520,39424)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(75520,39680)_2_label.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide1.mrxs\512\label\(75520,39936)_2_label.png


...

saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(116736,43008)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(116736,43264)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(116992,41728)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(116992,41984)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(116992,42240)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(116992,42496)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(116992,42752)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(116992,43008)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(116992,43264)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(117248,41472)_2_

saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(120832,38912)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(120832,39168)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(121088,36608)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(121088,36864)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(121088,37120)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(121088,37376)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(121088,37632)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(121088,37888)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(121088,38144)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(121088,38400)_2_

saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(123136,35072)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(123136,35328)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(123136,35584)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(123136,35840)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(123136,36096)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(123136,36352)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(123136,36608)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(123136,36864)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(123136,37120)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(123392,35840)_2_

saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(149248,38144)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(149248,38400)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(149504,37120)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(149504,37376)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(149504,37632)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(149504,37888)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(149504,38144)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(149504,38400)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(149504,38656)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(149504,38912)_2_

saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(151808,38144)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(151808,38400)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(151808,38656)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(151808,38912)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(151808,39168)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(151808,39424)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(151808,39680)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(151808,39936)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(152064,37376)_2_image.png
saving C:\Users\User\temp\slice\patches\temp\slides\slide5.mrxs\512\image\(152064,37632)_2_

In [11]:
# We have just saved both labels and images.
# Now we can load both labels and images from data store as one data-flow: load_named_ndarrays(data_path)
# But it is common case to load labels and images separately, so here is example.
from ndarray_persist.ndarray_persist_utils import load_named_ndarrays
named_labels = load_named_ndarrays(data_path, name_pattern=f'.*/{grid_length}/label/.*')
named_images = load_named_ndarrays(data_path, name_pattern=f'.*/{grid_length}/image/.*')
named_labels, named_images = list(named_labels), list(named_images)
print_named_ndarrays_info(named_labels)
print_named_ndarrays_info(named_images)

arrays count: 1407
1-st array (name,shape): ('temp/slides/slide1.mrxs/512/label/(63232,41728)_2_label.png', (512, 512, 3))
arrays count: 1407
1-st array (name,shape): ('temp/slides/slide1.mrxs/512/image/(63232,41728)_2_image.png', (512, 512, 3))


In [ ]:
# Lets plot (patch_image, patch_label) tuples
from common_matplotlib.core import plot_named_ndarrays_tuples_by_batches
image_tuples=zip(named_images,named_labels)
plot_named_ndarrays_tuples_by_batches(image_tuples, ncols=10, tuples_per_plot=20)